In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error,mean_absolute_error,root_mean_squared_error
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd
import re
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score
import xarray as xr


# 加载数据
df=pd.read_csv("prepare_2014-2022.csv",index_col=0)
a12=pd.read_csv("E:\\2023\\weather\\main\\O3finish\\allO3_2013.csv",index_col=0)
#a12=a12[a12['time'] == 990558]
a12= a12.reset_index(drop=True)
print(a12)

In [ ]:
df2=pd.read_csv("prepare_2000-2012.csv",index_col=0)
# 获取df1的列顺序
df_columns = df.columns.tolist()

# 将df2的列按照df1的列顺序进行重排
df2 = df2[df_columns]
df2.iloc[:, 3] = df2.iloc[:, 3].replace(-9999, 0)
df = df[~df.isin([ -9999.0]).any(axis=1)]
df2 = df2[~df2.isin([ -9999.0]).any(axis=1)]

In [ ]:
new_names = {col: re.sub(r'[^A-Za-z0-9_]+', '', col) for col in df.columns}
new_n_list = list(new_names.values())
# [LightGBM] Feature appears more than one time.
new_names = {col: f'{new_col}_{i}' if new_col in new_n_list[:i] else new_col for i, (col, new_col) in enumerate(new_names.items())}
gdf=df.copy()
gdf = gdf.rename(columns=new_names)
y = gdf.iloc[:, [0,3]]

gdf2=df2.copy()
gdf2 = gdf2.rename(columns=new_names)
y2 = gdf2.iloc[:, [0,3]]
print(gdf,gdf2)

gdf.drop(df.columns[3], axis=1, inplace=True)
gdf2.drop(df2.columns[3], axis=1, inplace=True)

# gdf.drop(['longitude', 'latitude'], axis=1, inplace=True)
# gdf2.drop(['longitude', 'latitude'], axis=1, inplace=True)
X = gdf
X_less=X.iloc[:,:]
X_less=X_less.iloc[:,:]
X2 = gdf2
X_less2=X2.iloc[:,:]
X_less2=X_less2.iloc[:,:]

# X_train, X_test, y_train, y_test = train_test_split(X_less.iloc[:,1:], y, test_size=0.2,random_state=42)

X_test=X_less2
X_test=X_test.iloc[:,1:]
X_train = X_less
X_train = X_train.iloc[:,1:]
y_test=y2
y_test=y_test.iloc[:,1]
y_train = y
y_train=y_train.iloc[:,1]

X_train.drop(['longitude', 'latitude'], axis=1, inplace=True)
X_test.drop(['longitude', 'latitude'], axis=1, inplace=True)

# 转换为Dataset数据格式
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [ ]:
# 参数
params = {
    'task': 'train',
    'boosting_type': 'gbdt',  # 设置提升类型
    'objective': 'regression',  # 回归任务
    'metric': {'l2', 'auc'},  # 评估函数
    'num_leaves': 20000,  # 叶子节点数
    'learning_rate': 0.1,  # 学习速率
    'feature_fraction': 0.95,  # 建树的特征选择比例
    'bagging_fraction': 0.95,  # 建树的样本采样比例
    'bagging_freq': 2,  # k 意味着每 k 次迭代执行bagging
    'verbose': 1 , # <0 显示致命的, =0 显示错误 (警告), >0 显示信息
    'device': 'gpu'
}

# 模型训练
gbm = lgb.train(params, lgb_train, num_boost_round=20, valid_sets=lgb_eval)

# 模型保存
gbm.save_model('model.txt')

# 模型加载
gbm = lgb.Booster(model_file='model.txt')

# 模型预测
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

In [ ]:
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)


In [ ]:
X_test=X_less2
X_train = X_less
y_test=y2
y_test=y_test.iloc[:,1]
y_train = y
y_train=y_train.iloc[:,1]


#X_train, X_test, y_train, y_test = train_test_split(X_less.iloc[:,:], y, test_size=0.2,random_state=42)


y_more=X_test.iloc[:,[0,1,2,3]]
y_more = y_more.reset_index(drop=True) 


y_pred_frame = pd.DataFrame({'O3_pred': y_pred})
y_pred_frame=y_pred_frame.reset_index(drop=True)
y_test_frame=  pd.DataFrame({"O3_true":y_test})
y_test_frame=y_test_frame.reset_index(drop=True)
result_df = pd.concat([y_more, y_pred_frame], axis=1)
result_df = result_df.drop(result_df.columns[3], axis=1)
original_df=pd.concat([y_more, y_test_frame], axis=1)
original_df = original_df.drop(original_df.columns[3], axis=1)

In [ ]:
import pandas as pd
import xarray as xr


# Assuming 'exact_time' is in hours since '1900-01-01 00:00:00.0'
base_time = pd.Timestamp('1900-01-01 00:00:00')

# First, ensure 'exact_time' is treated as numeric (hours since base_time)
result_df['exact_time'] = pd.to_numeric(result_df['exact_time'])

# Calculate the actual datetime from hours since base_time
result_df['exact_time'] = base_time + pd.to_timedelta(result_df['exact_time'], unit='h')

# Set up the dimensions and variable for xarray
ds = result_df.set_index(['exact_time', 'latitude', 'longitude']).to_xarray()

# Save the Dataset as a NetCDF file
output_path = ".//output//"+"result_2000-2012"+".nc"
ds.to_netcdf(output_path)

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr

# 假设nc文件路径为file1.nc, file2.nc, file3.nc
file_paths = [".//output//"+"result_1981-1990"+".nc",".//output//"+"result_1991-2000"+".nc",".//output//"+"result_2000-2012"+".nc"]

# 指定的经纬度点
target_longitude = 120.0  # 示例经度
target_latitude = 30.0   # 示例纬度

# 存储O3_pred值的列表
o3_values = []

# 存储时间标签
times = []

# 读取nc文件并提取指定位置的O3_pred数据
for file_path in file_paths:
    # 打开nc文件
    dataset = xr.open_dataset(file_path)
    
    # 提取时间、经度和纬度的维度信息
    time_values = dataset['exact_time'].values
    
    # 提取指定经纬度点的O3_pred数据
    o3_value = dataset['O3_pred'].sel(longitude=target_longitude, latitude=target_latitude, method='nearest').values
    
    # 记录O3_pred值和时间
    o3_values.append(o3_value)
    
    # 添加所有时间值（假设每个文件的时间维度有多个时间点）
    times.extend(time_values)  # 使用extend来合并多个时间值

# 将O3_pred值展平
o3_values = [value for sublist in o3_values for value in sublist]

# 按时间顺序绘制O3_pred的值
plt.figure(figsize=(10, 6))
plt.plot(times, o3_values, marker='o', linestyle='-', color='b', label='O3_pred')
plt.title(f'O3_pred Values Over Time at Longitude={target_longitude} and Latitude={target_latitude}')
plt.xlabel('Time')
plt.ylabel('O3_pred Value')
plt.xticks(rotation=45)  # 如果时间标签过长，可以旋转
plt.grid(True)
plt.legend()
plt.tight_layout()  # 自动调整布局
plt.show()


In [ ]:
import netCDF4 as nc

# 打开.nc文件
dataset = nc.Dataset(output_path)

# 查看文件中的变量
variables = dataset.variables
for var_name in variables:
    var = dataset.variables[var_name]
    print(f'变量名: {var_name}')
    print(f'维度: {var.dimensions}')
    print(f'形状: {var.shape}')
    print(f'属性: {var.ncattrs()}')
    for attr_name in var.ncattrs():
        print(f'{attr_name}: {getattr(var, attr_name)}')

variable_names = list(dataset.variables.keys())

# 打印变量名列表
print(variable_names)

# 关闭文件
dataset.close()


# Load the NetCDF file
file_path = output_path # Replace with the path to your NetCDF file
ds = xr.open_dataset(file_path)

# Check the available 'exact_time' values
print(ds['exact_time'])

# Select a specific date (e.g., the first date in the dataset)
selected_date = ds['exact_time'].isel(exact_time=2999)  # Change index to select the desired date

# Extract the 'O3_true' data for the selected date
o3_data = ds['O3_pred'].sel(exact_time='1986-07-04T06:00:00.000000000')

# Create a plot of 'O3_true' for the selected date
plt.figure(figsize=(10, 6))
o3_data.plot(x='longitude', y='latitude')  # Use x as longitude and y as latitude
plt.title(f'O3_true on {selected_date.values}')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.colorbar(label='O3_pred')
plt.show()